## Preliminaries

In [ ]:
library(ape)
library(ggtree)
library(phangorn)
library(Quartet)
library(adephylo)

#### Function for displaying face-to-face plot

In [ ]:
plot_cophylo <- function(tree1, tree2) {
    cophyloplot(ladderize(tree1), ladderize(tree2),
                assoc = matrix(rep(tree2$tip.label, 2), ncol = 2),
                length.line = 4, space = 28, gap = 3);
}

#### Function to display RF and Quartet metrics

In [ ]:
display_metrics <- function(tree, ref_tree) {
    splitStatuses <- SplitStatus(tree, ref_tree)
    quartetStatuses <- QuartetStatus(tree, ref_tree)
    # the least, the better (by distance)
    print(paste("Robinson-Foulds: ", RawSymmetricDifference(splitStatuses)))
    print(paste("Robinson-Foulds (normalized)", RawSymmetricDifference(splitStatuses) / splitStatuses[, 'N']))
    # the greater, the better (by similarity)
    print(paste("Similarity by Quartets: ", SymmetricDifference(quartetStatuses)))
}

## Plotting and comparison of trees

#### Loading reference tree

In [ ]:
ref_tree <- read.tree("dummy_tree.nwk")

#### Rank vs. Reference

In [ ]:
rank_tree <- read.tree("results/rank_tree.nwk")

display_metrics(rank_tree, ref_tree)

#png("ref_vs_rank.png")
plot_cophylo(ref_tree, rank_tree)
#dev.off()

#### Rank-Indel vs. Reference

In [ ]:
rankindl_tree <- read.tree("results/rankindl_tree.nwk")

display_metrics(rankindl_tree, ref_tree)

#png("ref_vs_rankindl.png")
plot_cophylo(ref_tree, rankindl_tree)
#dev.off()

#### DCJ vs. Reference

In [ ]:
dcj_tree <- read.tree("results/dcj_tree.nwk")

display_metrics(dcj_tree, ref_tree)

#png("ref_vs_dcj.png")
plot_cophylo(ref_tree, dcj_tree)
#dev.off()